In [ ]:
!pip install tsplib95

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.21.0 requires tabulate>=0.9, but you have tabulate 0.8.10 which is incompatible.


In [ ]:
# Run 1: population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/pr226.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 1536442.86613202
Generation 2: Best Cost = 1536442.86613202
Generation 3: Best Cost = 1536442.86613202
Generation 4: Best Cost = 1536442.86613202
Generation 5: Best Cost = 1536442.86613202
Generation 6: Best Cost = 1536442.86613202
Best solution found: [ 61  42  92 162 166 126   2 146 216  25 159  97 183 158  27  85 102  79
  58  40  53 129 212  34  45 149  96 154 164 157  20  37  39 152 195 108
  32  31 179 141 106  28 119 116  60 117  73  75 222 121 112 223  95 210
  14 204 107  91  55  49  94  62  67  72 128  74  56  70 207  89  48 122
   7 132  77 217 221 190  33  19 186 165 171 100 148 208  16 160 153 104
 194 168  64 211 155 151  18 167 138  35  44  57  93  83  59  69 147 192
  29 127 202  24  10 110 205  82 181  21  47 105 133 113 124  99 185   0
  12  36 180  52 137 142 213  46  66   3  80  76  50 193 174 118 219 114
 200 220  63  30 136 187  71 103 125 143   6  26  23  43 224 173   4 135
 203 206  88 163  17  84   5 191 130 140 198   9 172 175 209 161

In [ ]:
# Run 2: population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/pr226.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 1565155.816312206
Generation 2: Best Cost = 1565155.816312206
Generation 3: Best Cost = 1565155.816312206
Generation 4: Best Cost = 1565155.816312206
Generation 5: Best Cost = 1565155.816312206
Generation 6: Best Cost = 1565155.816312206
Best solution found: [ 39 141 219  73 130  60  97  96 110 147  30  35   0 120 108 192 206 148
  89 210 222 107  77 149 197 202 205 163 195 144  85  74  40 186  38  65
 137 174 121 139  51 177 208  94 178  46 146  75  90 173 169 209 213 223
 179 135 126 125 113  72   6 187 166 211 151  13 198  57  32  79  92  61
 157  25  27 199  29 201 111   1 224 140 159  62 167 112 212 184  11  12
  49  66 115 123 225  14  83 172   5 134  33  50 164 200 181 214 136 165
 180   9 220  93  88  18  28 150 143 124 216  21 191 204 103 217  55  41
  26  52 175 171  87 183 215  42 105 102 153  95 170 152  56 145 160 138
 207 182  23  81   8  67   3 142 155  48 100  34  22 106 196 193 176 119
 127 168 218  24 161  36 221  20  53  86  43 117  54 104 2

In [ ]:
# Run 3: population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/pr226.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 1516856.7543992253
Generation 2: Best Cost = 1516856.7543992253
Generation 3: Best Cost = 1516856.7543992253
Generation 4: Best Cost = 1516856.7543992253
Generation 5: Best Cost = 1516856.7543992253
Generation 6: Best Cost = 1516856.7543992253
Best solution found: [110 136 201 216  43 120 146 165  48 147 109 127 206 210 197 214  13  62
 195 149 180 203 124 112 212 139 182 179  36   4  75  42  56  52 135  19
 178 103 156 162  92 133  22 170  51   9  82   1  15 107  29 184 183  88
  76 173  40  60 144 115 121  45 158  91  11 130 218 209  71 131  31 222
 116   5   0  99  72 161 223 169 145  78 211  35 186 128 102  39 154 140
  18 114 225 153 157 177 171 123 188  69  80  85 125   3 126 172 221  74
 217  86 213 202 204 105   7 106  58 151  96 198 190  59 181 142  38  25
  47 129 166 164  65  55  89 152  34   2  30 101 215  23  21 191 208 148
 167 134 199 200 138 219 104  57   6   8  66  93  33  98 122 185 174  32
 224  50  54 176 117  94  49  24  70  37  16  14 143

In [ ]:
# Run 4: population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/pr226.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 1559753.3103234048
Generation 2: Best Cost = 1559753.3103234048
Generation 3: Best Cost = 1559753.3103234048
Generation 4: Best Cost = 1559753.3103234048
Generation 5: Best Cost = 1559753.3103234048
Generation 6: Best Cost = 1559753.3103234048
Best solution found: [175 131 171 212 176 161 170 125 201   6  54  46   8  22  80 112 219  59
  63 149 105 145  33 191 186  96 155 215 222 111 124 153 150 189 108  48
 129 220  34  97  16 187  84  31  13  90 154  43 195  78  62 224  57  64
  18  53  98   4 182 138  56 209 205 164  81  24  30  32 135  93  38 126
 203  74   3 140 166 158  40 202 213  99 183  91 211 107  15 194 156  11
 163 160  49 102 193 178  95  60 136 162 196 188  12  75 157 180  77  28
 116  87 133  66  69 109 101  45 106 115 139 144  82 172  94 130  58   9
  14  89 128 104  10 142 197 216  86  19  72 179 210 114 214 217  71 221
   0 218   1  23  20 198 117 190 147 127 173  92 177  36  26  37  29  67
  65  17  41 143  27 200 151 132 113  52 174  50  61

In [ ]:
# Run 5: population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/pr226.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 1519016.1291651642
Generation 2: Best Cost = 1519016.1291651642
Generation 3: Best Cost = 1519016.1291651642
Generation 4: Best Cost = 1519016.1291651642
Generation 5: Best Cost = 1519016.1291651642
Generation 6: Best Cost = 1519016.1291651642
Best solution found: [201 193 224 120  11   0  25 178 195  18 172 112 145  32 138 128  77  23
  72  85  65  70  78 155 143  62 132 179 162 117 123 106  74 202  98  30
  41  20 156 160  29  48 176  92 163   8  63 134  69 136 192 183 147  22
 111  80  84  73 152 129  51  36  42  58 109  54  95 205 209 169  87 194
 187 170 122 168  64 161   6 126 186 197 110  57  89  27  90 153  10 107
 166 137 139  88  49   7  15  37 118 148  21 135  28 105  31  59 141 223
 203 140 165 175  79 211 216  99  56 157 100 133  61 222 214   1 208 173
 190 198  82  13 225  16 119 114  55  34  47  81  39 151 124  94  86  50
  24 220  12 146 167   3 101 158  66 215 206 121 191 116 219  19 149 182
  44   4 103 171 104  91 221   9  33 142 188  83  67

In [ ]:
# Run 6: population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/pr226.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 1550925.7354074253
Generation 2: Best Cost = 1550925.7354074253
Generation 3: Best Cost = 1550925.7354074253
Generation 4: Best Cost = 1550925.7354074253
Generation 5: Best Cost = 1550925.7354074253
Generation 6: Best Cost = 1550925.7354074253
Best solution found: [ 77 129  12 212 104  76  29  59 136 220 160 176 103  17  61 211 101 106
  20 115  64  66 187 180 135   7  78  42 202  36  43  96  79   1  44 164
  97   2 105 221 209 207 120 171 150 184  48  71 201 122   0  37  90 131
  74 141 111 181  57  84 113  14 100 168 166 216 203  13 215 188 162  40
  69  63 217  19  83   9 189 185 192 183  82 169  38 156  31 198  86 165
 125  33 132 178 205 138 107 116  25  75 148  49 140  95  27 143 157  50
  98  94  70  87  60  41  47 167   3  30 222 199 179 126  72 154  99 119
 206  11 109  26  10  80 127 210 186 174 153 137 134 190 193  53 121 112
  23  51  45 196 139 214   4 149  88 158 204   5  65  92  24 117 218  39
  32  34 177 124 145  81 208 223 225 133 130  55  22

In [ ]:
# Run 7: population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/pr226.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 1551055.3015078239
Generation 2: Best Cost = 1551055.3015078239
Generation 3: Best Cost = 1551055.3015078239
Generation 4: Best Cost = 1551055.3015078239
Generation 5: Best Cost = 1551055.3015078239
Generation 6: Best Cost = 1551055.3015078239
Best solution found: [ 91 106  34  57   1 172 142 133 119 149  72 171 187 116 114 192 208  28
  77 200 162  18   6  19 194  61  45 169 121 216 123 130  22  71  87 173
  32  90 108 204  15  84 110  14 211  27  56 146 122  21  36  66  10 136
  96 198  50 107 163 183  20 118 127  48  17  76 212  58 219 175  67  99
  13  89 143 145  53  49 152 205 217 220 166 218  92 170 151 109 148 222
  81 155 137 213 197  24 147 128 144  82 125 188 207 139 190 214 101 140
 196  85 184  41  63  35  29 120 206  70   8   2 221  78  43 201  42  46
  73 100 153 154 168 131 124 195 132  74 134  11  62   9  52 115   3 104
 191 203  60  79 177  97   7  93  88  37  86 164 189 178 181 174  26  38
 135 138 176 129   4 105 210 167 223  51 160 165  23

In [ ]:
# Run 8: population_size=300, max_gen=6, elite_count=20, mutation_prob=1
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=300, max_gen=6, elite_count=20, mutation_prob=1):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/pr226.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=300, max_gen=6, elite_count=20, mutation_prob=1)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 1529388.8451824293
Generation 2: Best Cost = 1529388.8451824293
Generation 3: Best Cost = 1529388.8451824293
Generation 4: Best Cost = 1529388.8451824293
Generation 5: Best Cost = 1529388.8451824293
Generation 6: Best Cost = 1529388.8451824293
Best solution found: [137 145 191 109 221  55  67 213 135 106  40 139  95  97  59 111 198 125
 112 169 121  64 115 116  17 107 119 140 204  26  79 100 189   0 207 190
 136 209 126  89 217  25 201 171 222 196  22 105 162 214 123  82  47 181
  12 141  21  96 113 152 199 211 216  90  28 103   4  66 150 158  75   2
 166 120  68   7   6  87  99 176  44  58 163  49  56 208 192  57  27  35
  65  85  34  76 122   3 104 173 164  98  91 210 155  16 108 142 143  93
 138  92  46  88  36 114 203 110  37 206  10  30  52  33  61  19 101 197
 174 160 180 219 224  60 178  39  71  18 218 172  11 131  74   8 205 220
  80 187  45  41 134  15 185 117 146 102  23 212 167 188 153  83 202 149
 151  78  73 179 124  53 118  51  13  54  69 182 132

In [ ]:
# Run 9: population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/pr226.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 1515398.0616271184
Generation 2: Best Cost = 1515398.0616271184
Generation 3: Best Cost = 1515398.0616271184
Generation 4: Best Cost = 1515398.0616271184
Generation 5: Best Cost = 1515398.0616271184
Generation 6: Best Cost = 1515398.0616271184
Best solution found: [136 206 107 150  98 165 158 151  81  93 141  70  43  82  69  33  10  31
  15 221 143 112 142  45 121 188 100 183   6 109 108 164   7  18 149 117
  59 154  41 190 172   8 170 203 104  32  16 192  83  71 210 219 157 171
  79   4  66 220  19 144 138  46 118  72 120  42 224 115 179 123 131  25
  67  11 102  86  92  39 198  78 130 135 216 196  58 127 222 132  13 217
 194 197  65  34  99 145 156  94 159  37 110 213 114 214 155  64  53  50
  27  85  14 191 201  28  52 129 195   5 152 207 162 178 215  36 169 126
 182 134 185 166 128 111 205 208 218  76  68 225  60  62  80 133 116 200
 193  74   0 105  20  91 101  54  26  49 139 184  29  87 175  95 146   2
 163  73 153  55 202 106  17  75 211 125 209 173 176

In [ ]:
# Run 10: population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/pr226.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 1530462.177344559
Generation 2: Best Cost = 1530462.177344559
Generation 3: Best Cost = 1530462.177344559
Generation 4: Best Cost = 1530462.177344559
Generation 5: Best Cost = 1530462.177344559
Generation 6: Best Cost = 1530462.177344559
Best solution found: [ 24 171  40 156 105  12  96   5  35  60  73 132  89  27 109  75 204 115
 113 120 217  26  32 151 141 138 148 147 158 165  70 146 155 200 144  77
 121 179 173 187 199 103 112 176 193 192 177   3 128 169  74 127 195  94
  50 140   8  63  55  54 209 116 145   0  11  61 180 164 160   9  15 142
  82   2 213  66 104  17  23 100 136 203 216 111 102 107 153  86  41  29
 125  97  19  37  57 123  39  43  20 201 219  72 214 185 119 190 150 222
  91 124  49  36   6 205 168  13 126 206 215 198 211  71 129 159 166 175
 223  93  53  51  69 189 181  33  80  18 154   1 184 162  45  95  58  52
 161  21 183 225  98 101  34 130 188  65  85  84 172  46 170 224  99  87
  38 221 191  44   7  92 196  22 143  16  67 212 194   4  